In [7]:
import json

# Specify the path to your JSON file
json_file_path = "excretion.json"

# Read the JSON file and store its content in original_data
with open(json_file_path, "r") as infile:
    original_data = json.load(infile)

# Now original_data contains the content of your JSON file
print(original_data)


[{'context': 'Animals accumulate various substances such as ammonia, urea, uric acid, carbon dioxide, water, and ions like Na+, K+, Clâ€“, phosphate, and sulphate, either through metabolic activities or excess ingestion. These substances must be partially or completely removed from the body. In this chapter, you will learn about the mechanisms of eliminating these substances, with a focus on common nitrogenous wastes. Ammonia, urea, and uric acid are the primary forms of nitrogenous wastes excreted by animals. Ammonia is highly toxic and requires a large amount of water for elimination, while uric acid, being the least toxic, can be excreted with minimal water loss.', 'qas': [{'id': '00001', 'is_impossible': False, 'question': 'What are some substances that animals accumulate?', 'answers': [{'text': 'Animals accumulate substances such as ammonia, urea, uric acid, carbon dioxide, water, and ions like Na+, K+, Clâ€“, phosphate, and sulphate.', 'answer_start': 19}]}, {'id': '00002', 'is_i

In [8]:
# Convert data to T5 format
t5_formatted_data = []

for entry in original_data:
    context = entry.get("context", "")
    qas = entry.get("qas", [])
    for qa in qas:
        input_text = f"context: {context}"
        target_text = f"Question: {qa['question']} Answer: {qa['answers'][0]['text']}"
        t5_formatted_data.append({"input_text": input_text, "target_text": target_text})

# Save the T5 formatted data to a file
with open("t5_formatted_data.json", "w") as outfile:
    json.dump(t5_formatted_data, outfile, indent=2)

print("Data conversion completed. T5 formatted data saved to 't5_formatted_data.json'.")


Data conversion completed. T5 formatted data saved to 't5_formatted_data.json'.


In [5]:
import json

# Specify the path to your JSON file
json_file_path = "t5_formatted_data.json"

# Read the JSON file and store its content in original_data
with open(json_file_path, "r") as infile:
    excretion_data= json.load(infile)

# Now original_data contains the content of your JSON file
print(excretion_data)


[{'input_text': 'context: Animals accumulate various substances such as ammonia, urea, uric acid, carbon dioxide, water, and ions like Na+, K+, Clâ€“, phosphate, and sulphate, either through metabolic activities or excess ingestion. These substances must be partially or completely removed from the body. In this chapter, you will learn about the mechanisms of eliminating these substances, with a focus on common nitrogenous wastes. Ammonia, urea, and uric acid are the primary forms of nitrogenous wastes excreted by animals. Ammonia is highly toxic and requires a large amount of water for elimination, while uric acid, being the least toxic, can be excreted with minimal water loss.', 'target_text': 'Question: What are some substances that animals accumulate? Answer: Animals accumulate substances such as ammonia, urea, uric acid, carbon dioxide, water, and ions like Na+, K+, Clâ€“, phosphate, and sulphate.'}, {'input_text': 'context: Animals accumulate various substances such as ammonia, ur

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import pandas as pd

class QGDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data.iloc[idx]['input_text']
        target_text = self.data.iloc[idx]['target_text']
        
        input_ids = self.tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)
        target_ids = self.tokenizer.encode(target_text, return_tensors='pt', max_length=128, truncation=True)

        return {
            'input_ids': input_ids.flatten(),
            'labels': target_ids.flatten()
        }

def fine_tune_t5(train_data, model, tokenizer, epochs=3, learning_rate=5e-5, batch_size=4, device='cuda'):
    train_dataset = QGDataset(train_data, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    model.to(device)
    model.train()
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        total_loss = 0
        tqdm_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch')

        for batch in tqdm_loader:
            inputs = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            tqdm_loader.set_postfix({'loss': total_loss / len(tqdm_loader)})

    # Save the fine-tuned model
    model.save_pretrained('./fine_tuned_model')

if __name__ == "__main__":
    # Load your dataset
    dataset = excretion_data

    df = pd.DataFrame(dataset)

    # Load pre-trained T5 model and tokenizer
    checkpoint = 't5-small'
    tokenizer = T5Tokenizer.from_pretrained(checkpoint)
    model = T5ForConditionalGeneration.from_pretrained(checkpoint)

    # Fine-tune the model
    fine_tune_t5(df, model, tokenizer, epochs=3, learning_rate=5e-5, batch_size=4, device='cuda')


AssertionError: Torch not compiled with CUDA enabled

In [2]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [3]:
train_data

[('context: Animals accumulate various substances such as ammonia, urea, uric acid, carbon dioxide, water, and ions like Na+, K+, Clâ€“, phosphate, and sulphate, either through metabolic activities or excess ingestion. These substances must be partially or completely removed from the body. In this chapter, you will learn about the mechanisms of eliminating these substances, with a focus on common nitrogenous wastes. Ammonia, urea, and uric acid are the primary forms of nitrogenous wastes excreted by animals. Ammonia is highly toxic and requires a large amount of water for elimination, while uric acid, being the least toxic, can be excreted with minimal water loss.',
  'Question: What are some substances that animals accumulate? Answer: Animals accumulate substances such as ammonia, urea, uric acid, carbon dioxide, water, and ions like Na+, K+, Clâ€“, phosphate, and sulphate.'),
 ('context: Animals accumulate various substances such as ammonia, urea, uric acid, carbon dioxide, water, an